# Transport Calculation Preparation

- Take ADCP data from chosen site (C1, C2, C3)   
- low pass filter (lanzcos) and rotate arbitrary degrees   

then or C1,C2,C3 - pull out NE (+/- 30 deg?)

In [192]:
#load file and filter

import xarray as xa
import pandas as pd
import numpy as np

In [193]:
%matplotlib inline

In [194]:
## up a few directories there is a filters folder

from inspect import getsourcefile
import os.path as path, sys
current_dir = path.dirname(path.abspath(getsourcefile(lambda:0)))
sys.path.insert(0, current_dir[:current_dir.rfind(path.sep)])
import filters.lanzcos as lanzcos  # Replace "my_module" here with the module name.
sys.path.pop(0)

'/Users/bell/Programs/Python/EcoFOCI_Jupyter_Notebooks'

**Load ADCP data with CF convention timewords**

In [195]:
filein='/Users/bell/ecoraid/2018/Moorings/18ckp3a/erddap/final_data/18ckp3a_wcp_vel.cf.nc'
xadata = xa.load_dataset(filein)

In [196]:
xadata

<xarray.Dataset>
Dimensions:    (depth: 7, lat: 1, lon: 1, time: 8779)
Coordinates:
  * time       (time) datetime64[ns] 2018-08-12T20:00:00 ... 2019-08-13T14:00:00
  * depth      (depth) float32 8.0 12.0 16.0 20.0 24.0 28.0 32.0
  * lat        (lat) float32 71.82485
  * lon        (lon) float32 166.05847
Data variables:
    v_1206     (time, depth, lat, lon) float32 -4.4028864 ... 4.965061
    u_1205     (time, depth, lat, lon) float32 11.991021 12.376356 ... 7.654944
    w_1204     (time, depth, lat, lon) float32 -0.1 -0.1 -0.2 ... 0.1 0.0 0.0
    Werr_1201  (time, depth, lat, lon) float32 0.1 -0.8 0.6 ... -0.5 0.3 0.6
Attributes:
    CREATION_DATE:        January 27, 2020 19:31 UTC
    COMPOSITE:            1
    INST_TYPE:            wcp
    DATA_CMNT:            1705.VEL
    EPIC_FILE_GENERATOR:  nc_epic2udunits_time.py V0.4.0
    PROG_CMNT01:          
    EDIT_CMNT01:          
    WATER_DEPTH:          47.0
    MOORING:              18CKP-3A
    WATER_MASS:           B
    EXPERIMENT:           
    PROJECT:              
    History:               \nJanuary 27, 2020 19:31 UTC EPIC two time-word ke...

In [197]:
#to do it pandas way
fdata=pd.DataFrame()
for depths in xadata.depth:
    udata = xadata.sel(depth=depths).u_1205[:,0,0].to_pandas()
    vdata = xadata.sel(depth=depths).v_1206[:,0,0].to_pandas()
    
    data = pd.concat([udata, vdata], axis=1)
    data.columns = ['udata','vdata']
    
    data['udata'][data.udata > 1e34]=np.nan
    data['vdata'][data.vdata > 1e34]=np.nan
    data = data.resample('1H').mean()
    
    ## missing data fill limited by number of hours to linearly interpolate
    data['tempu'] = data['udata'].interpolate(method='time',limit=6)
    data['tempv'] = data['vdata'].interpolate(method='time',limit=6)

    # get index of missing data that wasn't filled
    # the remaining missing data will continue to be missing after analysis
    # this data and the edges of the timeseries need to have the effects of the filter window accounted for

    missing_index_u = np.isnan(data.tempu)
    missing_index_v = np.isnan(data.tempv)
    data['tempu'] = data['udata'].interpolate(method='time')
    data['tempv'] = data['vdata'].interpolate(method='time')
    
    #filter data with rudimentary matlab script
    data['ufdata'] = lanzcos.lanzcos(data.tempu.values,1,35)+data['udata'].mean()
    data['vfdata'] = lanzcos.lanzcos(data.tempv.values,1,35)+data['vdata'].mean()
    data['ufdata'][missing_index_u] = np.nan
    data['vfdata'][missing_index_v] = np.nan
    
    #remove boundaries of large gaps

    """
    for index, row in data[np.isnan(data[var2filter_name+'_f35'])].iterrows():
            data[var2filter_name+'_f35'][index-datetime.timedelta(hours=17):index+datetime.timedelta(hours=17)] = np.nan
    """
    #remove start/end boundaries
    for index in range(0,17):
        data.loc[data.index[index]]['ufdata'] =np.nan
        data.loc[data.index[index]]['vfdata'] =np.nan

    for index in range(len(data)-17,len(data)):
        data.loc[data.index[index]]['ufdata'] =np.nan
        data.loc[data.index[index]]['vfdata'] =np.nan
        
    data.drop(columns=['tempu','tempv'],inplace=True)
    
    # rotate uv
    print("rotating vectors")
    # when rotating vectors - positive(+) rotation is equal to ccw of the axis (cw of vector)
    #                      - negative(-) rotation is equal to cw of the axis (ccw of the vector)
    angle_offset_rad = np.deg2rad(45)
    uprime = data["ufdata"] * np.cos(angle_offset_rad) + data["vfdata"] * np.sin(
        angle_offset_rad
    )
    vprime = -1.0 * data["udata"] * np.sin(angle_offset_rad) + data["vfdata"] * np.cos(
        angle_offset_rad
    )

    fdata["vfrdata_"+str(int(depths.values))] = vprime
    fdata["ufrdata_"+str(int(depths.values))] = uprime
    print(str(int(depths.values)))

/Users/bell/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/bell/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/bell/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/bell/anaconda2/envs/py37/lib/python3.7/site-p

rotating vectors
8
rotating vectors
12
rotating vectors
16
rotating vectors
20
rotating vectors
24
rotating vectors
28
rotating vectors
32


In [198]:
fdata.to_csv(filein.split('.cf.nc')[0]+'_35hrfiltered_45degrotated.csv')
data.to_csv(filein.split('.cf.nc')[0]+str(int(depths.values))+'.csv')